In [ ]:
import boto3

client = boto3.client('ec2')

In [ ]:
my_session = boto3.session.Session()
my_region = my_session.region_name
print my_region

In [ ]:
# AMIs are unique per region. Here we will look up our AMI based on our region
amis = {}
amis['us-east-1'] = 'ami-0b33d91d'
amis['us-east-2'] = 'ami-446f3521'
amis['us-west-1'] = 'ami-9fadf8ff'
amis['us-west-2'] = 'ami-7abc111a'
amis['eu-west-1'] = 'ami-a1491ad2'
amis['ca-central-1'] = 'ami-ebed508f'

ami_id = amis[my_region]
print ami_id

In [ ]:
#Create security group
response = client.create_security_group(
    GroupName='web_sg',
    Description='Use for launching public web service'
)

webSgID = response['GroupId']
print 'created security group ', webSgID

In [ ]:
# We'll add some rules for inbound traffic - note that security groups are stateful:
# if you allow inbound traffic, the response based on that inbound traffic is allowed
# outbound without an explicit rule granting that. This is in contrast with network
# access control lists which are stateless - you have to pair them if you want 
# to enable output based on allowed input

# By default, all traffic is blocked: we explicitly enable the traffic we want to allow.

# Allow ingress on port 80
response = client.authorize_security_group_ingress(
    GroupId=webSgID,
    IpProtocol='tcp',
    FromPort=80,
    ToPort=80,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
# Allow ingress for ssh
response = client.authorize_security_group_ingress(
    GroupId=webSgID,
    IpProtocol='tcp',
    FromPort=22,
    ToPort=22,
    CidrIp='0.0.0.0/0'
)

print response

In [ ]:
import os

keyName = os.environ["KEYNAME"]
print keyName

In [ ]:
user_data = \
"""#!/bin/bash
yum update -y
yum install httpd -y
cd /var/www/html
echo "<html><h1>Hello cloud gurus</h1></html>" > /var/www/html/index.html
service httpd start"""

In [ ]:
# Launch instance - default VPC, default subnet, etc.
launch_response = client.run_instances(
    KeyName = keyName,
    DryRun=False,
    ImageId=ami_id,
    MinCount=1,
    MaxCount=1,
    InstanceType='t2.micro',
    UserData=user_data,
    SecurityGroups=[
        'web_sg'
    ],
)


instanceId = launch_response['Instances'][0]['InstanceId']

In [ ]:
# Wait for instance
print 'Waiting for launch of ', instanceId
waiter = client.get_waiter('instance_running')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)
print 'Launch complete'

In [ ]:
# We can get the status for our instance, including the System Status 
# (can AWS get network packets to the instance?) and the Instance
# Status (is the operating system accepting traffic)
response = client.describe_instance_status(
    InstanceIds=[
        instanceId,
    ]
)

print response

In [ ]:
# Get some cloud watch metrics
from datetime import datetime, timedelta
cw = boto3.client('cloudwatch')
response = cw.get_metric_statistics(
    Namespace='AWS/EC2',MetricName='CPUUtilization',
    StartTime=datetime.utcnow() - timedelta(days=1) ,
    EndTime=datetime.utcnow(), Period=300,
    Statistics=['Average'], Unit='Percent',
    Dimensions=[
        {'Name': 'InstanceId', 'Value': instanceId}
    ])


print(response)

response = cw.get_metric_statistics(
    Namespace='AWS/EC2',MetricName='DiskWriteOps',
    StartTime=datetime.utcnow() - timedelta(days=1) ,
    EndTime=datetime.utcnow(), Period=300,
    Statistics=['Average'], Unit='Count',
    Dimensions=[
        {'Name': 'InstanceId', 'Value': instanceId}
    ])


print(response)


response = cw.get_metric_statistics(
    Namespace='AWS/EC2',MetricName='NetworkPacketsIn',
    StartTime=datetime.utcnow() - timedelta(days=1) ,
    EndTime=datetime.utcnow(), Period=300,
    Statistics=['Average'], Unit='Count',
    Dimensions=[
        {'Name': 'InstanceId', 'Value': instanceId}
    ])


print(response)

In [ ]:
# Get information about the attached volume, which in this case is the boot volume
response = client.describe_volumes(
    Filters=[
        {
            'Name': 'attachment.instance-id',
            'Values': [
                instanceId
            ]
        },
    ]
)

print response

vol_az = response['Volumes'][0]['AvailabilityZone']
print 'volume az:', vol_az

In [ ]:
# Create a second volume (cold storage)
vol_response = client.create_volume(
    AvailabilityZone=vol_az,
    Size=500,
    VolumeType='sc1',
    Encrypted=False
)

print vol_response

In [ ]:
# Attach the volume to our EC2 instance
vol_id = vol_response['VolumeId']
print vol_id


attach_resp = client.attach_volume(
    VolumeId=vol_id,
    InstanceId=instanceId,
    Device='xvdf'
)

print attach_resp

In [ ]:
# Describe volumes again after attaching our new volume
# Get information about the attached volume, which in this case is the boot volume
response = client.describe_volumes(
    Filters=[
        {
            'Name': 'attachment.instance-id',
            'Values': [
                instanceId
            ]
        },
    ]
)

print response

In [ ]:
response = client.describe_instances(
InstanceIds=[
        instanceId,
    ]
)

print response['Reservations'][0]['Instances'][0]['PublicDnsName']

In [ ]:
# For the next notebook, ssh to the instance then do the following
do_this ="""
cd /
mkdir myfileserver
lsblk
file -s /dev/xvdf
mkfs -t ext4 /dev/xvdf
mount /dev/xvdf /myfileserver
cd /myfileserver
create some files
ls
lsblk
umount /dev/xvdf
"""

In [ ]:
# Detach the volume
det_response = client.detach_volume(
    VolumeId=vol_id,
    InstanceId=instanceId,
    Device='xvdf'
)

print det_response

In [ ]:
# Note: if we were taking a snapshot of a boot volume we'd stop the instance first
# to make sure we had a consistent snapshot
response = client.create_snapshot(
    VolumeId=vol_id,
    Description='my snappin shot'
)

print response

In [ ]:
# Delete the volume
response = client.delete_volume(
    VolumeId=vol_id
)

print response

In [ ]:
response = client.describe_snapshots(
    
    Filters=[
        {
            'Name': 'description',
            'Values': [
                'my snappin shot'
            ]
        },
    ]
)

print response

snapshot_id = response['Snapshots'][0]['SnapshotId']
print snapshot_id

In [ ]:
# Create a new volume from the snapshot, this time make it a general purpose volume
vol_response = client.create_volume(
    AvailabilityZone=vol_az,
    SnapshotId=snapshot_id,
    Size=500,
    VolumeType='gp2',
    Encrypted=False
)

print vol_response

In [ ]:
# Attach the volume created from the snapshot to our EC2 instance
vol_id = vol_response['VolumeId']
print vol_id


attach_resp = client.attach_volume(
    VolumeId=vol_id,
    InstanceId=instanceId,
    Device='xvdf'
)

print attach_resp

In [ ]:
# Now ssh to the box, mount the volume, and see the files created previously

In [ ]:
# Detach the volume
det_response = client.detach_volume(
    VolumeId=vol_id,
    InstanceId=instanceId,
    Device='xvdf'
)

print det_response

In [ ]:
# Delete the volume
response = client.delete_volume(
    VolumeId=vol_id
)

print response

In [ ]:
response = client.delete_snapshot(
    SnapshotId=snapshot_id
)

print response

In [ ]:
# Terminate the EC2 Instance
response = client.terminate_instances(
    InstanceIds=[
        instanceId,
    ]
)

print response

In [ ]:
print 'wait for it...'
waiter = client.get_waiter('instance_terminated')
waiter.wait(
    InstanceIds=[
        instanceId,
    ]
)
print 'proceed'

In [ ]:
# Clean up security group
response = client.delete_security_group(
    GroupId=webSgID
)

print response